In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
df_train = pd.read_csv("/kaggle/input/titanic/train.csv")
df_test = pd.read_csv("/kaggle/input/titanic/train.csv")
df_train.head(10)

In [ ]:
def preprocess(df):

    def normalize(x):
        for i in ",()[].\"'":
            x = x.replace(i, '') # Замена посторонних символов на пробелы в имени
        return x

    def ticket_numbers(x):
        return x.split(' ')[-1] # Разделяем билет на буквенную и числовую части, берем числовую .она в конце.

    def ticket_item(x):
        items = x.split(' ') # Берем буквенную часть билета, если она есть, если нет, то в строку присваиваем None
        if len(items) == 1:
            return 'None'
        return ' '.join(items[0:-1])
    
    df['Name'] = df['Name'].apply(normalize)
    df['Ticket_Number'] = df['Ticket'].apply(ticket_numbers)
    df['Ticket_item'] = df['Ticket'].apply(ticket_item)
    return df

                     
preprocessed_train_df = preprocess(df_train)
preprocessed_serving_df = preprocess(df_test)

preprocessed_train_df.head(5)



       

In [ ]:
df_train.isnull().sum() # Смотрим кол-во пропусков, в колонке Возраст 177 пропусков, в "Номер кабины" 687, что сильно больше половины, принимаю решение удалить его.

In [ ]:
df_train.drop(columns = ['Cabin'],axis = 1, inplace=True) # Удаление колонки Cabin


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

Survived_sex = sns.countplot(x='Survived', hue='Sex', data=df_train)
Survived_sex.axes.set_title("Зависимость выживаемости от пола", fontsize=14) # Как мы видим мужчин погибло больше

In [ ]:
SibSp_Survived = sns.countplot(x='Survived', hue='SibSp', data=df_train) # Зависимость выживаемости от кол-ва близких (Братья,сестры,Муж, Жена)


In [ ]:
Parch_Survived = sns.countplot(x='Survived', hue='Parch', data=df_train) # Зависимость выживаемости от кол-ва близких (Мать, Отец, дочь, сын, падчерица, пасынок)

In [ ]:
Count_survived = sns.countplot(x=df_train['Survived'])
Count_survived.legend(bbox_to_anchor=(1.02, 1), loc='upper left', borderaxespad= 0 ) # Сколько погибло

In [ ]:
p_class_survived = sns.countplot(x = 'Survived', hue = 'Pclass', data=df_train) # Видим что погибло больше пассажирова 3 класса каюты, т.е самого низкого

In [ ]:
age_survived = sns.boxplot(data=df_train, x='Survived', y="Age");

In [ ]:
from sklearn.neighbors import KNeighborsClassifier # Решил использовать метод ближайших K соседей, для классификации.
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score

features = ['Pclass', 'Sex', 'SibSp', 'Parch']

X_train = pd.get_dummies(df_train[features])
y_train = df_train['Survived']
X_test = pd.get_dummies(df_test[features])
y_test = df_test['Survived']

clf = KNeighborsClassifier()
parametrs = {'n_neighbors': range(1, 20)}
cv = KFold(n_splits=5, shuffle=True, random_state=10)
grid = GridSearchCV(clf, parametrs, cv=cv)
grid.fit(X_train, y_train)
predicted = grid.predict(X_test)
accuracy = accuracy_score(y_test, predicted)
accuracy # Пока что получили 80%


In [ ]:
output = pd.DataFrame({'PassengerId':df_test.PassengerId, 'Survived':predicted})
output.to_csv('my_submission.csv', index=False)